In [1]:
#!apt-get install -y libgl1-mesa-dev
!pip3 install --upgrade pip
!pip3 install torch==1.8.1+cu111 torchvision==0.9.1+cu111 torchaudio==0.8.1 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html


In [2]:
!pip3 install numpy
!pip3 install pandas
!pip3 install multiprocessing
!pip3 install botocore
!pip3 install boto3
!pip3 install scikit-build
!pip3 install cmake
!pip3 install opencv-python

  Using cached multiprocessing-2.6.2.1.tar.gz (108 kB)
    ERROR: Command errored out with exit status 1:
     command: /usr/bin/python3 -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-xi7dmijd/multiprocessing_335e754fad424943ab07166e42874c1d/setup.py'"'"'; __file__='"'"'/tmp/pip-install-xi7dmijd/multiprocessing_335e754fad424943ab07166e42874c1d/setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base /tmp/pip-pip-egg-info-xel4kw3m
         cwd: /tmp/pip-install-xi7dmijd/multiprocessing_335e754fad424943ab07166e42874c1d/
    Complete output (6 lines):
    Traceback (most recent call last):
      File "<string>", line 1, in <module>
      File "/tmp/pip-install-xi7dmijd/multiprocessing_335e754fad424943ab07166e42874c1d/setup

In [3]:
#確認
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name())
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

True
GeForce RTX 3070


In [4]:
import cv2
import logging
import pandas as pd
import logging
import os
from urllib import request
def http_download(url, path):
    with request.urlopen(url) as f:
        with open(path, "wb") as fout:
            buf = f.read(1024)
            while buf:
                fout.write(buf)
                buf = f.read(1024)

from multiprocessing import Pool, Manager
import functools

def log_counts(values):
    for k, count in values.value_counts().iteritems():
        logging.warning(f"{k}: {count}/{len(values)} = {count/len(values):.2f}.")

import boto3
from botocore import UNSIGNED
from botocore.config import Config
import botocore
s3 = boto3.client('s3', config=Config(signature_version=UNSIGNED))
def download(bucket, root, retry, counter, lock, path):
    i = 0
    src = path
    dest = f"{root}/{path}"
    logging.warning("start downloading :" +  str(dest))
    while i < retry:
        try:
            if not os.path.exists(dest):
                s3.download_file(bucket, src, dest)
            else:
                logging.info(f"{dest} already exists.")
            with lock:
                counter.value += 1
                if counter.value % 100 == 0:
                    logging.warning(f"Downloaded {counter.value} images.")
            return
        except botocore.exceptions.ClientError as e:
            if e.response['Error']['Code'] == "404":
                logging.warning(f"The file s3://{bucket}/{src} does not exist.")
                return
            i += 1
            logging.warning(f"Sleep {i} and try again.")
            time.sleep(i)
    logging.warning(f"Failed to download the file s3://{bucket}/{src}. Exception: {e}")

def batch_download(bucket, file_paths, root, num_workers=4, retry=1):
    with Pool(num_workers) as p:
        m = Manager()
        counter = m.Value('i', 0)
        lock = m.Lock()
        download_ = functools.partial(download, bucket, root, retry, counter, lock)
        p.map(download_, file_paths)
"""
#直列に変更
def batch_download(bucket, file_paths, root, num_workers=4, retry=1):
    m = Manager()
    counter = m.Value('i', 0)
    lock = m.Lock()
    for file_path in file_paths:
        download(bucket, root, retry, counter, lock, file_path)
"""

"\n#直列に変更\ndef batch_download(bucket, file_paths, root, num_workers=4, retry=1):\n    m = Manager()\n    counter = m.Value('i', 0)\n    lock = m.Lock()\n    for file_path in file_paths:\n        download(bucket, root, retry, counter, lock, file_path)\n"

In [5]:

#download open-images, make my own dataset!!!!
bucket = "open-images-dataset"
class_description_file = './class-descriptions-boxable.csv' #bofore download CLI 
class_descriptions = pd.read_csv(class_description_file,names=["id", "ClassName"])
#learn bottle data
class_names = ["Bottle", "Box", "Ball", "Person", "Bird", "Car", "Clock", "Laptop", "Traffic sign", "Jacket", "Plate", "Shirt", "Lamp"]
group_filters = []
percentages = [0.4, 1.0, 0.7, 0.03, 0.15, 0.07, 1.0, 0.5, 1.0, 0.2, 1.0, 0.4, 1.0]
#TODO 怪しい
for class_name in class_names:
    group_filters.append("")
excluded_images = set()
class_descriptions = class_descriptions[class_descriptions['ClassName'].isin(class_names)]

image_files = []
object_dir = "./images_openimages"
os.makedirs(object_dir, exist_ok=True)
for dataset_type in ["train", "validation", "test"]:
    image_dir = os.path.join(object_dir, dataset_type)

    #csvのダウンロード(バカ長い)
    os.makedirs(image_dir, exist_ok=True)
    annotation_file = f"{image_dir}/{dataset_type}-annotations-bbox.csv"
    if os.path.exists(annotation_file): #check
        os.remove(annotation_file)
    url = f"https://storage.googleapis.com/openimages/2018_04/{dataset_type}/{dataset_type}-annotations-bbox.csv"
    logging.warning(f"Download {url}.")
    http_download(url, annotation_file)
    logging.warning(f"Read annotation file {annotation_file}")
    annotations = pd.read_csv(annotation_file)
    annotations = pd.merge(annotations, class_descriptions,left_on="LabelName", right_on="id",how="inner")
    annotations = annotations.loc[annotations['IsDepiction'] != 1, :] #?

    filtered = []
    for class_name, group_filter, percentage in zip(class_names, group_filters, percentages):
        sub = annotations.loc[annotations['ClassName'] == class_name, :]
        excluded_images |= set(sub['ImageID'].sample(frac=1 - percentage))

        if group_filter == '~group':
            excluded_images |= set(sub.loc[sub['IsGroupOf'] == 1, 'ImageID'])
        elif group_filter == 'group':
            excluded_images |= set(sub.loc[sub['IsGroupOf'] == 0, 'ImageID'])
        filtered.append(sub)

    annotations = pd.concat(filtered)
    annotations = annotations.loc[~annotations['ImageID'].isin(excluded_images), :]

    annotations = annotations.sample(frac=1.0)

    logging.warning(f"{dataset_type} bounding boxes size: {annotations.shape[0]}")
    logging.warning("Approximate Image Stats: ")
    log_counts(annotations.drop_duplicates(["ImageID", "ClassName"])["ClassName"])
    logging.warning("Label distribution: ")
    log_counts(annotations['ClassName'])

    logging.warning(f"Shuffle dataset.")


    sub_annotation_file = f"{object_dir}/sub-{dataset_type}-annotations-bbox.csv"
    if os.path.exists(sub_annotation_file): #check
        os.remove(sub_annotation_file)
    logging.warning(f"Save {dataset_type} data to {sub_annotation_file}.")
    annotations.to_csv(sub_annotation_file, index=False)
    image_files.extend(f"{dataset_type}/{id}.jpg" for id in set(annotations['ImageID']))

logging.warning(f"Start downloading {len(image_files)} images.")
batch_download(bucket, image_files, object_dir, 4, 3)
logging.warning("Task Done.")
